In [1]:
# Please, always put this cell on top of any Notebook you make.
# It replaces the former cells with Google Drive mounting and change directory.
# Using the code in this cell ensures that the Notebook is easy to run on Google Colab and on Jupyter Lab alike.
import os

def init_google_colab(gpath = '/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3'):
    """If in Google Colab, switch to relevant working directory

    Returns:
    --------
    bool    True if running in Google Colab, False if not.
    """
    in_google_colab = False
    try:
        from google.colab import drive
        in_google_colab = True
    except:
        pass
    if in_google_colab:
        drive.mount("/content/gdrive")
        os.chdir(gpath)
    return in_google_colab

init_google_colab('/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


True

In [2]:
import pandas as pd
from pandas import read_excel
import nltk
from nltk.corpus import wordnet as wn
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('omw-1.4')
nltk.download('stopwords')
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import pandas as pd
import nltk
import string
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = read_excel("assignment3_data.xlsx")
labels = read_excel("assignment3_gold_labels.xlsx")


In [4]:
data.head()

,What qualities do you think are necessary to be the prime minister of India?
0,"Intelligence, integrity, decisiveness"
1,"Honesty, passion, determination, teamwork"
2,U have to be well qualified
3,"Honesty, good work ethic, understanding of sys..."
4,Common sense


In [5]:


data.columns = ["Data"]

#Extracting the words

l1 = []
for index, row in data.iterrows():
    words = row["Data"].split(" ")
    for j in words:
        l1.append(j.lower())

#Removing punctuations and stopwords
        
def punc(text):
    exclude = set(string.punctuation)
    text = [''.join(x for x in y if x not in exclude) for y in text]
    text = [x for x in text if x not in stopwords.words('english')]
    return text

l1 = punc(l1)

#Creating the temporary vocabulary of nouns and adjectives

POS = ['JJ','JJR','JJS','NN','NNS','NNP','NNPS']
temp_vocab = sorted([word for (word,pos) in nltk.pos_tag(l1) if pos in POS])
df1 = pd.DataFrame(columns = ["Words", "Frequency"])
c1 = Counter(temp_vocab)
df1["Words"] = c1.keys()
df1["Frequency"] = c1.values()
df1 = df1.sort_values(by = ["Frequency"], ascending = False)
df1.to_csv(r"df1.csv", sep = ',')


In [6]:
#Finding the synsets and derivationally related forms containing nouns and adjectives

def syn(text):
    s = []
    for w in text:
        i = wn.synsets(str(w),pos = 'n')
        j = wn.synsets(str(w), pos = 'a')
        if i!=[]:
            s.append(i)
        if j !=[]:
            s.append(j)
    return s

s1 =syn(temp_vocab)

d1 = []
for sy in s1:
    for a in sy:
        for k in a.lemmas():
            for z in k.derivationally_related_forms():
                    d1.append(z.name())

s2 = syn(d1)

#Creating the main vocabulary

vocab = []
for i in s1:
    for j in i:
        vocab.append(j)
for i in s2:
    for j in i:
        vocab.append(j)
vocab = sorted(set(vocab))



In [7]:
#Calculating the similarities between the synsets

def sim(mat):
    thres = 0.8
    l = []
    for i in mat:
        m = []
        for j in mat:
            si = wn.wup_similarity(i,j)
            if(si == None):
                continue
            if(si > thres):
                m.extend((i,j))
        l.append(m)
    return l

S = sim(vocab)

#List of non-empty clusters

S1 = []
for i in S:
    if len(set(i)) >= 1:
        S1.append(list(set(i)))
       
#Assigning clusters to each word in the vocabulary

dict1 = {}
for i in set(temp_vocab):
    for j in S1:
        for k in j:
            if(i in k.lemma_names()):
                dict1[i] = sorted(list(set(j)))

#Combining words with same clusters

dict2 = {}
for key, value in dict1.items():
    dict2[tuple(value)] = key

dict1 = {}
for k, v in dict2.items():
    dict1[v] = list(k)

#Extracting the names for each clusters

p = []
q = []
for x, y in dict1.items():
    a = []
    p.append(x)
    for i in y:
        a.append(i.lemma_names())
    q.append(sum(a, []))

r = []
for i in q:
    r.append(sorted(list(set(i))))

t = []
for i in list(dict1.values()):
    z = []
    for j in i:
        z.append(j.name())
    t.append(z)

for i in range(len(r)):
    r[i].append(t[i])

for i in r:
    for j in i:
        if isinstance(j,list):
            for k in j:
                i.append(k)
            i.remove(j)



In [8]:
#Creating a dataframe and displaying the frequency of each cluster

df = pd.DataFrame(columns = ["Word","Frequency","Clusters"])
df["Word"] = p
df["Clusters"] = r

frq = []
for x, y in df.iterrows():
    f = 0
    for i in range(len(y["Clusters"])):
        f += temp_vocab.count(y.Clusters[i])
    frq.append(f)

df["Frequency"] = frq
df = df.sort_values(by = ["Frequency"], ascending = False)


In [9]:
df[:10]

,Word,Frequency,Clusters
3,ability,17,"[ability, accomplishment, acquirement, acquisi..."
6,politics,15,"[aid, assist, assistance, attention, campaign,..."
77,integrity,10,"[honestness, honesty, integrity, honesty.n.01,..."
72,equality,10,"[equality, equation, equivalence, leadership, ..."
31,morality,8,"[honestness, honesty, honor, honorableness, ho..."
19,dedication,7,"[allegiance, commitment, compromise, dedicatio..."
27,basic,5,"[basic, commodity, facility, fashion, good, in..."
13,president,5,"[Chief_Executive, President, President_of_the_..."
40,good,4,"[benefit, good, goodness, soundness, welfare, ..."
16,county,4,"[country, county, land, state, country.n.02, c..."


In [10]:
df[:10].to_csv("extracted_words_wordnet.txt")